#  Predicting Boston Housing Prices

## Import required packages.

In [1]:
from pathlib import Path

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.neural_network import MLPRegressor 
from sklearn.preprocessing import StandardScaler

from dmba import regressionSummary

%matplotlib inline
import matplotlib.pylab as plt

## Upload data set for analysis. Explore, clean, and pre-process data. 

In [2]:
# Create data frame from the original data set.  
boston_df = pd.read_csv('BostonHousing.csv')

# Determine dimensions of dataframe. 
print('BostonHousing Dimensions:', boston_df.shape)

BostonHousing Dimensions: (506, 14)


In [3]:
# Display the column names.
print('')
print('Original Column Titles')
boston_df.columns


Original Column Titles


Index(['CRIME', 'ZONE', 'INDUST', 'CHAR RIV', 'NIT OXIDE', 'ROOMS', 'AGE',
       'DISTANCE', 'RADIAL', 'TAX', 'ST RATIO', 'LOW STAT', 'MVALUE',
       'C MVALUE'],
      dtype='object')

In [4]:
# Make column titles (variable names) as one word and 
# without blank. 
boston_df.columns = [s.strip().replace(' ', '_') for s in boston_df.columns]
print('Converted One-Word Titles')
boston_df.columns

Converted One-Word Titles


Index(['CRIME', 'ZONE', 'INDUST', 'CHAR_RIV', 'NIT_OXIDE', 'ROOMS', 'AGE',
       'DISTANCE', 'RADIAL', 'TAX', 'ST_RATIO', 'LOW_STAT', 'MVALUE',
       'C_MVALUE'],
      dtype='object')

In [5]:
# Display column data types in the data frame for regression analysis.
print('BostonHousing Data Types')
print(boston_df.dtypes)

# Convert object variables  into dummy variables.
# Use drop_first=True to drop the first dummy variable.
boston_df = pd.get_dummies(boston_df, prefix_sep='_', 
                            drop_first=True)

# Disply updated data types. 
print('')
print('Converted BostonHousing Data Types')
print(boston_df.dtypes)


BostonHousing Data Types
CRIME        float64
ZONE         float64
INDUST       float64
CHAR_RIV      object
NIT_OXIDE    float64
ROOMS        float64
AGE          float64
DISTANCE     float64
RADIAL         int64
TAX            int64
ST_RATIO     float64
LOW_STAT     float64
MVALUE       float64
C_MVALUE      object
dtype: object

Converted BostonHousing Data Types
CRIME           float64
ZONE            float64
INDUST          float64
NIT_OXIDE       float64
ROOMS           float64
AGE             float64
DISTANCE        float64
RADIAL            int64
TAX               int64
ST_RATIO        float64
LOW_STAT        float64
MVALUE          float64
CHAR_RIV_Y        uint8
C_MVALUE_Yes      uint8
dtype: object


In [7]:
boston_df.head()

,CRIME,ZONE,INDUST,NIT_OXIDE,ROOMS,AGE,DISTANCE,RADIAL,TAX,ST_RATIO,LOW_STAT,MVALUE,CHAR_RIV_Y,C_MVALUE_Yes
0,0.00632,18.0,2.31,0.538,6.575,65.2,4.0900,1,296,15.3,4.98,24.0,0,0
1,0.02731,0.0,7.07,0.469,6.421,78.9,4.9671,2,242,17.8,9.14,21.6,0,0
2,0.02729,0.0,7.07,0.469,7.185,61.1,4.9671,2,242,17.8,4.03,34.7,0,1
3,0.03237,0.0,2.18,0.458,6.998,45.8,6.0622,3,222,18.7,2.94,33.4,0,1
4,0.06905,0.0,2.18,0.458,7.147,54.2,6.0622,3,222,18.7,5.33,36.2,0,1


## Neural network model for BostonHousing  data. 

In [8]:
# Identify outcome and predictors. Create data partitioning 
# and scale the data using StandardScaler()from scikit-learn 
# libray. Display original and scaled predictors for training 
# partition. 

# Develop outcome. 
outcome = 'MVALUE'

# Develop predictor variables. 
predictors = [s for s in boston_df.columns if s not in outcome]

# Identify X and y variables for BostonHousing data set and 
# partition data using 70% of records for training and 30% 
# for validation (test_size=0.3). 
X = boston_df[predictors]
y = boston_df[outcome]
train_X, valid_X, train_y, valid_y = train_test_split(X, y, 
                            test_size=0.3, random_state=1)

# Display the first 5 records of Toyota Corolla training 
# partition's predictors. 
print('Predictors for Training Partition')
print(train_X.head(5))

# Scale input data (predictors) for training  and validation 
# partitions using StandardScaler().
sc_X = StandardScaler()
train_X_sc = sc_X.fit_transform(train_X)
valid_X_sc = sc_X.transform(valid_X)

# Develop a data frame to display scaled predictors for 
# training partition. Round scaled values to 3 decimals.
# Add coloumn titles to data frame.
train_X_sc_df = np.round(pd.DataFrame(train_X_sc), decimals=3)                            
train_X_sc_df.columns=['CRIME', 'ZONE', 'INDUST', 'NIT_OXIDE', 
      'ROOMS', 'AGE', 'DISTANCE', 'RADIAL', 'TAX', 'ST_RATIO', 
      'LOW_STAT',  'CHAR_RIV_Y','C_MVALUE_Yes']

# Display scaled predictors for training partition.
print()
print('Scaled Predictors for Training Partition')
print(train_X_sc_df.head(5))

Predictors for Training Partition
       CRIME  ZONE  INDUST  NIT_OXIDE  ROOMS   AGE  DISTANCE  RADIAL  TAX  \
13   0.62976   0.0    8.14      0.538  5.949  61.8    4.7075       4  307   
61   0.17171  25.0    5.13      0.453  5.966  93.4    6.8185       8  284   
377  9.82349   0.0   18.10      0.671  6.794  98.8    1.3580      24  666   
39   0.02763  75.0    2.95      0.428  6.595  21.8    5.4011       3  252   
365  4.55587   0.0   18.10      0.718  3.561  87.9    1.6132      24  666   

     ST_RATIO  LOW_STAT  CHAR_RIV_Y  C_MVALUE_Yes  
13       21.0      8.26           0             0  
61       19.7     14.44           0             0  
377      20.2     21.24           0             0  
39       18.3      4.32           0             1  
365      20.2      7.12           0             0  

Scaled Predictors for Training Partition
   CRIME   ZONE  INDUST  NIT_OXIDE  ROOMS    AGE  DISTANCE  RADIAL    TAX  \
0 -0.366 -0.484  -0.462     -0.147 -0.440 -0.251     0.412  -0.646 -0.60

In [12]:
# Use MLPRegressor() function to train neural network model.
# Apply: 
# (a) default input layer with the number of nodes equal 
#     to number of predictor variables (13); 
# (b) default single hidden layer with 10 nodes; 
# (c) default output layer with one outcome variable (Price);
# (d) optimization function solver = 'lbfgs', 
#     which is applied for small data sets for better 
#     performance and fast convergence. For large data sets, 
#     apply default solver = 'adam' optimization function;
# (e) model is fit with scaled predictors and regular outcome
#     in training partition.
boston_nn = MLPRegressor(hidden_layer_sizes=(10), 
                solver='lbfgs', max_iter=10000, random_state=1)
boston_nn.fit(train_X_sc, train_y)

# Display network structure with the final values of 
# intercepts (Theta) and weights (W).
print('Final Intercepts for Boston Housing Neural Network Model')
print(boston_nn.intercepts_)

print()
print('Network Weights for Boston Housing Neural Network Model')
print(boston_nn.coefs_)

Final Intercepts for Boston Housing Neural Network Model
[array([ 0.13292439, -5.12357884, -3.10562394, -0.12894016, -1.37297091,
       -1.39127175, -1.36315706,  0.21571478,  3.79757179, -0.10013371]), array([3.09174442])]

Network Weights for Boston Housing Neural Network Model
[array([[ 0.54597464,  1.34058253,  0.02213642,  0.15030378, -2.38288775,
        -0.20435814, -0.99299133, -0.28532504, -0.03440528,  0.2705021 ],
       [ 0.82100873,  0.13165897,  0.40264055,  0.96120668, -2.19159579,
        -0.0245981 , -0.27159266, -0.87856117, -0.21737456,  1.90352302],
       [ 1.28901735,  0.85759979, -0.00391884, -0.27625905, -0.20055893,
         0.63708174, -0.37733494, -0.32682421,  0.27764283, -1.85053288],
       [ 1.33865853,  0.71723071, -0.35772379, -3.06606683,  2.10898016,
         0.2086184 , -0.03642794,  1.56647258, -3.00257008,  0.95609551],
       [ 2.64986499,  0.69712263, -0.36456842, -0.98406414,  0.29351945,
         1.14293934, -0.60364141, -0.0251318 ,  0.043875

In [13]:
# Make 'MVALUE' predictions for validation set using  
# Boston Housing neural network model. 

# Use boston_nn model to predict 'MVALUE' outcome
# for validation set.
mvalue_pred = np.round(boston_nn.predict(valid_X_sc), decimals=2)

# Create data frame to display prediction results for
# validation set. 
mvalue_pred_result = pd.DataFrame({'Actual': valid_y, 
                'Prediction': mvalue_pred, 'Residual': valid_y-mvalue_pred})

print('Predictions for Toyota Price for Validation Partition')
print(mvalue_pred_result.head())

Predictions for Toyota Price for Validation Partition
     Actual  Prediction  Residual
307    28.2       29.89     -1.69
343    23.9       25.55     -1.65
47     16.6       20.72     -4.12
67     22.0       20.64      1.36
362    20.8       24.44     -3.64


In [14]:
# Neural network model accuracy measures for training and
# validation partitions. 

# Identify and display neural network model accuracy measures 
# for training partition.
print('Accuracy Measures for Training Partition for Neural Network')
regressionSummary(train_y, boston_nn.predict(train_X_sc))

# Identify and display neural network accuracy measures 
# for validation partition.
print()
print('Accuracy Measures for Validation Partition for Neural Network')
regressionSummary(valid_y, boston_nn.predict(valid_X_sc))

Accuracy Measures for Training Partition for Neural Network

Regression statistics

                      Mean Error (ME) : -0.0030
       Root Mean Squared Error (RMSE) : 1.6154
            Mean Absolute Error (MAE) : 1.1592
          Mean Percentage Error (MPE) : -0.9159
Mean Absolute Percentage Error (MAPE) : 6.2577

Accuracy Measures for Validation Partition for Neural Network

Regression statistics

                      Mean Error (ME) : -0.6335
       Root Mean Squared Error (RMSE) : 3.9271
            Mean Absolute Error (MAE) : 2.7627
          Mean Percentage Error (MPE) : -5.8985
Mean Absolute Percentage Error (MAPE) : 14.6733


## Grid search for Boston Housing neural network model.

In [15]:
# Identify grid search parameters. 
param_grid = {
    'hidden_layer_sizes': list(range(2, 20)), 
}

# Utilize GridSearchCV() to identify the best number 
# of nodes in the hidden layer. 
gridSearch = GridSearchCV(MLPRegressor(solver='lbfgs', max_iter=10000, random_state=1), 
                          param_grid, cv=5, n_jobs=-1, return_train_score=True)
gridSearch.fit(train_X_sc, train_y)

# Display the best score and best parament value.
print(f'Best score:{gridSearch.best_score_:.4f}')
print('Best parameter: ', gridSearch.best_params_)


Best score:0.8877
Best parameter:  {'hidden_layer_sizes': 2}


In [16]:
# Use MLPRegressor() function to train improved neural network model
# based on grid search. 

# Apply: 
# (a) default input layer with the number of nodes equal 
#     to number of predictor variables (13); 
# (b) default single hidden layer with 2 nodes; 
# (c) default output layer with one outcome variable (Price);
# (d) optimization function solver = 'lbfgs', 
#     which is applied for small data sets for better 
#     performance and fast convergence. For large data sets, 
#     apply default solver = 'adam' optimization function;
# (e) model is fit with scaled predictors and regular outcome
#     in training partition.
boston_nn_imp = MLPRegressor(hidden_layer_sizes=(2), 
                solver='lbfgs', max_iter=10000, random_state=1)
boston_nn_imp.fit(train_X_sc, train_y)

# Display network structure with the final values of 
# intercepts (Theta) and weights (W).
print('Final Intercepts for Boston Housing Neural Network Model')
print(boston_nn_imp.intercepts_)

print()
print('Network Weights for Boston Housing Neural Network Model')
print(boston_nn_imp.coefs_)


Final Intercepts for Boston Housing Neural Network Model
[array([-5.77798235,  9.23251124]), array([6.42319947])]

Network Weights for Boston Housing Neural Network Model
[array([[-0.3032743 , -1.12974977],
       [-0.83116756,  0.01885639],
       [ 3.41885652, -0.19609333],
       [-0.87643445, -0.34832306],
       [-1.38687676,  2.44231364],
       [ 0.03476532, -1.00136674],
       [-0.27505135, -1.03085599],
       [ 3.41072744, -0.42693217],
       [ 1.71442508, -1.53029944],
       [-1.30596363, -0.71167013],
       [-1.27352383, -0.47077159],
       [ 0.23975211,  0.12005179],
       [ 3.21479073,  1.48260895]]), array([[2.27967747],
       [1.50200358]])]


In [13]:
# Neural network model accuracy measures for training and
# validation partitions of improved neural network. 

# Identify and display neural network model accuracy measures 
# for training partition.
print('Accuracy Measures for Training Partition for Improved Neural Network')
regressionSummary(train_y, boston_nn_imp.predict(train_X_sc))

# Identify and display neural network accuracy measures 
# for validation partition.
print()
print('Accuracy Measures for Validation Partition for Improved Neural Network')
regressionSummary(valid_y, boston_nn_imp.predict(valid_X_sc))

Accuracy Measures for Training Partition for Improved Neural Network

Regression statistics

                      Mean Error (ME) : -0.0011
       Root Mean Squared Error (RMSE) : 2.6988
            Mean Absolute Error (MAE) : 2.0685
          Mean Percentage Error (MPE) : -1.8598
Mean Absolute Percentage Error (MAPE) : 10.6435

Accuracy Measures for Validation Partition for Improved Neural Network

Regression statistics

                      Mean Error (ME) : 0.1360
       Root Mean Squared Error (RMSE) : 3.0161
            Mean Absolute Error (MAE) : 2.2839
          Mean Percentage Error (MPE) : -2.7418
Mean Absolute Percentage Error (MAPE) : 12.0915
